# Decode stimuli/block/licks from neural activity - DR

In [ ]:
#Import packages
import pandas as pd
import h5py
import numpy as np
import scipy as sp
import scipy.signal as sg
import scipy.stats as st
import xarray as xr
import os
import matplotlib.pyplot as plt
from matplotlib import patches
import ast
from sklearn import svm
import glob
import re
import pickle

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedKFold

from DR_analysis_utils import Session, makePSTH, make_neuron_time_trials_tensor, compute_smoothed_response_rate

%matplotlib notebook

In [ ]:
#Set paths to experiment folders
main_path = [
    r"\\allen\programs\mindscope\workgroups\dynamicrouting\PilotEphys\Task 2 pilot\DRpilot_626791_20220815\processed",
    r"\\allen\programs\mindscope\workgroups\dynamicrouting\PilotEphys\Task 2 pilot\DRpilot_626791_20220816\processed",
    r"\\allen\programs\mindscope\workgroups\dynamicrouting\PilotEphys\Task 2 pilot\DRpilot_626791_20220817\processed",
    r"\\allen\programs\mindscope\workgroups\dynamicrouting\PilotEphys\Task 2 pilot\DRpilot_636766_20230123\processed", 
    r"\\allen\programs\mindscope\workgroups\dynamicrouting\PilotEphys\Task 2 pilot\DRpilot_636766_20230124\processed", 
    r"\\allen\programs\mindscope\workgroups\dynamicrouting\PilotEphys\Task 2 pilot\DRpilot_636766_20230125\processed", 
    r"\\allen\programs\mindscope\workgroups\dynamicrouting\PilotEphys\Task 2 pilot\DRpilot_636766_20230126\processed", 
    r"\\allen\programs\mindscope\workgroups\dynamicrouting\PilotEphys\Task 2 pilot\DRpilot_644864_20230130\processed",
    r"\\allen\programs\mindscope\workgroups\dynamicrouting\PilotEphys\Task 2 pilot\DRpilot_644864_20230131\processed", 
    r"\\allen\programs\mindscope\workgroups\dynamicrouting\PilotEphys\Task 2 pilot\DRpilot_644864_20230201\processed", 
    r"\\allen\programs\mindscope\workgroups\np-exp\PilotEphys\Task 2 pilot\DRpilot_644864_20230202\processed",
#     r"\\allen\programs\mindscope\workgroups\np-exp\PilotEphys\Task 2 pilot\DRpilot_644866_20230207\processed", 
#     r"Y:\DRpilot_644866_20230208\processed",
#     r"Y:\DRpilot_644866_20230209\processed",
#     r"Y:\DRpilot_644866_20230210\processed",
    r"Y:\DRpilot_644867_20230220\processed",
    r"Y:\DRpilot_644867_20230221\processed",
    r"Y:\DRpilot_644867_20230222\processed",
    r"Y:\DRpilot_644867_20230223\processed",
    r"Y:\DRpilot_649943_20230213\processed", 
    r"Y:\DRpilot_649943_20230214\processed",
    r"Y:\DRpilot_649943_20230215\processed",
    r"Y:\DRpilot_649943_20230216\processed",
]



## Load a single session

In [ ]:
sel_session=0
session=Session(path=main_path[sel_session]) 
session.assign_unit_areas()
session=compute_smoothed_response_rate(session)

In [ ]:
# trials table
session.trials

In [ ]:
# units table
session.good_units

In [ ]:
session.good_units['area'].value_counts()

In [ ]:
# spike times from one unit
session.spike_times[session.units.index[0]]

In [ ]:
# information about each frame
session.frames

In [ ]:
# loop through sessions and make unit xarrays
time_before_flash = 0.5
trial_duration = 1.5
bin_size = 0.001


# Make tensor (3-D matrix [units,time,trials])
trial_tensor = make_neuron_time_trials_tensor(session.good_units, session.spike_times, 
                                              session.trials,time_before_flash, trial_duration, 
                                              bin_size)

# make xarray
session.trial_da = xr.DataArray(trial_tensor, dims=("unit_id", "time", "trials"), 
                           coords={
                               "unit_id": session.good_units.index.values,
                               "time": np.arange(0, trial_duration, bin_size)-time_before_flash,
                               "trials": session.trials.index.values
                               })

### decode by area

In [ ]:
# session.good_units['area'].value_counts()

In [ ]:
#shorten the area names to better lump together units
#get rig of layers and/or sub-areas with dashes
area_short = []
for area in session.good_units['area']:
    if area=='N/A':
        short='N/A'
    elif area[:2]=='CA':
        short=area
    else:
        dig_ind=re.search(r"\d", area)
        dash_ind=re.search(r"-", area)
        if dig_ind!=None:
            short=area[:dig_ind.start()]
        elif dash_ind!=None:
            short=area[:dash_ind.start()]
        else:
            short=area
        
    area_short.append(short)
    
session.good_units['area_short']=area_short

In [ ]:
area_counts=session.good_units['area_short'].value_counts()
area_counts[area_counts>35].index

In [ ]:
session.trials

In [ ]:
# # grab data: make these [trials,neurons] matrices based on different time bins:
# trial_sel=session.trials.query('(trialStimID == "sound1" and trial_response == True)').index

# area_sel = area_counts[area_counts>25].index

# # grab the stimulus ids
# stim_ids = session.trials['trialStimID'][trial_sel].values
# # or, use block IDs
# block_ids = session.trials['trialstimRewarded'][trial_sel].values
# #or, use whether mouse responded
# trial_response = session.trials['trial_response'][trial_sel].values

# # choose what variable to predict
# pred_var = block_ids

# ### make sure equal # of trials per condition!
# conds = np.unique(pred_var)
# cond_count = np.zeros(len(conds))
# for ic,cc in enumerate(conds):
#     cond_count[ic]=np.sum(pred_var==cc)
    
# min_n_trials=np.min(cond_count).astype(int)

# subset_ind=[]

# for cc in conds:
#     cond_inds=np.where(pred_var==cc)[0]
#     subset_ind.append(np.random.choice(cond_inds,min_n_trials,replace=False))
    
# subset_ind=np.sort(np.hstack(subset_ind))

In [ ]:
# subset_ind

In [ ]:
#try more timepoints & split by areas

svc_results={}
binsize=0.1

# grab data: make these [trials,neurons] matrices based on different time bins:

hit_trials = session.trials.query(
    '(trialStimID == "vis1" and \
    trialstimRewarded == "vis1" and \
    trial_response == True and \
    vis_autoreward_trials == False) or \
    (trialStimID == "sound1" and \
    trialstimRewarded == "sound1" and \
    trial_response == True and \
    vis_autoreward_trials == False)').index

cr_trials = session.trials.query(
    '(trialStimID == "vis1" and \
    trialstimRewarded == "sound1" and \
    trial_response == False and \
    vis_autoreward_trials == False) or \
    (trialStimID == "sound1" and \
    trialstimRewarded == "vis1" and \
    trial_response == False and \
    vis_autoreward_trials == False)').index


trial_sel=np.sort(np.hstack([hit_trials,cr_trials]))

area_sel = area_counts[area_counts>30].index
# area_sel = ['all']

# grab the stimulus ids
stim_ids = session.trials['trialStimID'][trial_sel].values
# or, use block IDs
block_ids = session.trials['trialstimRewarded'][trial_sel].values
#or, use whether mouse responded
trial_response = session.trials['trial_response'][trial_sel].values

#choose what variable to predict
pred_var = block_ids

### make sure equal # of trials per condition!
conds = np.unique(pred_var)
cond_count = np.zeros(len(conds))
for ic,cc in enumerate(conds):
    cond_count[ic]=np.sum(pred_var==cc)
min_n_trials=np.min(cond_count).astype(int)
subset_ind=[]
for cc in conds:
    cond_inds=np.where(pred_var==cc)[0]
    subset_ind.append(np.random.choice(cond_inds,min_n_trials,replace=False))   
subset_ind=np.sort(np.hstack(subset_ind))
trial_sel=trial_sel[subset_ind]
pred_var=pred_var[subset_ind]

trnum=40
u_num=30

n_repeats=100

time_bins=np.arange(-0.2,0.6,binsize)

for aa in area_sel:
    if aa=='all':
        unit_sel = session.good_units.index.values
    else:
        unit_sel = session.good_units.query('area_short==@aa').index.values
    svc_results[aa]={}
    
    for tt,t_start in enumerate(time_bins[:-1]):
        svc_results[aa][tt]={}
        
        for nn in range(0,n_repeats):
            unit_subset = np.random.choice(unit_sel,u_num,replace=True)

            ### make sure equal # of trials per condition!
            subset_ind=[]
            conds = np.unique(pred_var)
            cond_count=[]

            for cc in conds:
                cond_inds=np.where(pred_var==cc)[0]
                subset_ind.append(np.random.choice(cond_inds,trnum,replace=False))   
            subset_ind=np.sort(np.hstack(subset_ind))

            sel_data = session.trial_da.sel(time=slice(t_start,time_bins[tt+1]),
                                            trials=trial_sel[subset_ind],
                                            unit_id=unit_subset).mean(dim='time').values

    #         sel_data = session.trial_da.sel(time=slice(t_start,time_bins[tt+1]),
    #                                         trials=trial_sel,unit_id=unit_sel).mean(dim='time').values

            X = sel_data.T
            y = pred_var[subset_ind].flatten()

            xtrain, xtest, ytrain, ytest = train_test_split(X, y, test_size=0.5, stratify=y)

            clf = svm.LinearSVC()
            clf.fit(xtrain, ytrain)

            ypred = clf.predict(xtest)

            cr_dict=classification_report(ytest, ypred, output_dict=True)
            cr_df=pd.DataFrame.from_dict(cr_dict)

            svc_results[aa][tt][nn]=cr_df
    
    print(aa+' done')
        

In [ ]:
hit_trials.shape

In [ ]:
# make into more plottable format

plot_results={}

timepoints=time_bins[1:]

areas=area_sel

for aa in areas:
    plot_results[aa]={}
#     plot_results[tt]['modality']=modality
    plot_results[aa]['vis1']=np.zeros((len(timepoints),n_repeats))
    plot_results[aa]['vis1'][:]=np.nan
    plot_results[aa]['vis2']=np.zeros((len(timepoints),n_repeats))
    plot_results[aa]['vis2'][:]=np.nan
    plot_results[aa]['aud1']=np.zeros((len(timepoints),n_repeats))
    plot_results[aa]['aud1'][:]=np.nan
    plot_results[aa]['aud2']=np.zeros((len(timepoints),n_repeats))
    plot_results[aa]['aud2'][:]=np.nan
    
    plot_results[aa]['True']=np.zeros((len(timepoints),n_repeats))
    plot_results[aa]['True'][:]=np.nan
    plot_results[aa]['False']=np.zeros((len(timepoints),n_repeats))
    plot_results[aa]['False'][:]=np.nan
#     for tt,tp in enumerate(timepoints):
#         plot_results[aa][tt]=[]
        

         
# diff table for each timepoint?
# row = session
# columns = ['modality','A_vis','A_aud','B_vis','B_aud','C_vis','C_aud','F_vis','F_aud',]

for aa in areas: #svc_results[ss]['results'].keys():
    if aa in svc_results.keys():
        for tt,tp in enumerate(timepoints):
            
            for nn in range(0,n_repeats):
                if 'vis1' in svc_results[aa][tt][nn].keys():
                    vis1_perf=np.mean(svc_results[aa][tt][nn]['vis1'][['precision','recall']].values)
                    plot_results[aa]['vis1'][tt,nn]=vis1_perf

                if 'vis2' in svc_results[aa][tt][nn].keys():
                    vis2_perf=np.mean(svc_results[aa][tt][nn]['vis2'][['precision','recall']].values)
                    plot_results[aa]['vis2'][tt,nn]=vis2_perf

                if 'sound1' in svc_results[aa][tt][nn].keys():
                    aud1_perf=np.mean(svc_results[aa][tt][nn]['sound1'][['precision','recall']].values)
                    plot_results[aa]['aud1'][tt,nn]=aud1_perf

                if 'sound2' in svc_results[aa][tt][nn].keys():
                    aud2_perf=np.mean(svc_results[aa][tt][nn]['sound2'][['precision','recall']].values)
                    plot_results[aa]['aud2'][tt,nn]=aud2_perf

                if 'True' in svc_results[aa][tt][nn].keys():
                    aud1_perf=np.mean(svc_results[aa][tt][nn]['True'][['precision','recall']].values)
                    plot_results[aa]['True'][tt,nn]=aud1_perf

                if 'False' in svc_results[aa][tt][nn].keys():
                    aud2_perf=np.mean(svc_results[aa][tt][nn]['False'][['precision','recall']].values)
                    plot_results[aa]['False'][tt,nn]=aud2_perf

#     plot_results[tt]=pd.DataFrame.from_dict(plot_results[tt])

In [ ]:
svc_results[aa][tt][nn]['vis1']

In [ ]:
# plot_results
np.std(np.hstack([plot_results[area]['vis1'],plot_results[area]['aud1']]),1)
# np.hstack([plot_results[area]['vis1'],plot_results[area]['aud1']]).shape

In [ ]:
fig,ax=plt.subplots(1,1,figsize=(6,4))



for aa,area in enumerate(plot_results.keys()):
    y=np.nanmean([np.mean(plot_results[area]['vis1'],1),np.mean(plot_results[area]['aud1'],1)],0)
    yerr=np.std(np.hstack([plot_results[area]['vis1'],plot_results[area]['aud1']]),1)/np.sqrt(n_repeats)
#     ax.plot(timepoints,np.mean([plot_results[area]['vis1'],plot_results[area]['aud1']],0))
    linex=ax.plot(timepoints,y)
    plt.fill_between(timepoints, y-yerr, y+yerr,
                    alpha=0.2, edgecolor=None, facecolor=linex[0].get_color())
    
#     ax[aa].plot(timepoints,plot_results[area]['aud1'])

ax.set_ylim([0.4,1.0])
ax.axvline(0,color='k')
# ax.axhline(0.25,color='k',linewidth=0.5,linestyle='--')
ax.axhline(0.5,color='k',linewidth=0.5,linestyle='--')
ax.set_title(session.metadata['mouseID']+' session '+str(session.metadata['ephys_session_num'])+
            '; n_units='+str(u_num)+'; n_trials='+str(trnum)+'; n_repeats='+str(n_repeats))
# ax.set_title(area)
# ax.set_xticklabels('')
ax.legend(plot_results.keys(),ncol=3)
ax.set_ylabel('decodability of context')
ax.set_xlabel('time from stimulus onset (s)')
fig.tight_layout()

In [ ]:
# fig,ax=plt.subplots(len(plot_results.keys()),1,figsize=(6,20))

# if type(ax)!=np.ndarray:
#     ax=[ax]

# for aa,area in enumerate(plot_results.keys()):
#     ax[aa].plot(timepoints,np.mean([plot_results[area]['vis1'],plot_results[area]['aud1']],0))
# #     ax[aa].plot(timepoints,plot_results[area]['aud1'])
#     ax[aa].set_ylim([-0.1,1.1])
#     ax[aa].axvline(0,color='k')
#     ax[aa].axhline(0.25,color='k',linewidth=0.5,linestyle='--')
#     ax[aa].axhline(0.5,color='k',linewidth=0.5,linestyle='--')
#     ax[aa].set_title(area)
#     if aa<len(plot_results.keys())-1:
#         ax[aa].set_xticklabels('')

# ax[aa].set_xlabel('time from stimulus onset (s)')
# fig.tight_layout()

## loop through experiments & plot

In [ ]:
trnum=30
u_num=20
n_repeats=100
binsize=0.1
time_bins=np.arange(-0.2,0.6,binsize)

for sel_session, path in enumerate(main_path):
    
    session=[]
    session=Session(path=path) 
    session.assign_unit_areas()
    session=compute_smoothed_response_rate(session)
    
    # loop through sessions and make unit xarrays
    time_before_flash = 0.25
    trial_duration = 1.0
    bin_size = 0.001

    # Make tensor (3-D matrix [units,time,trials])
    trial_tensor = make_neuron_time_trials_tensor(session.good_units, session.spike_times, 
                                                  session.trials,time_before_flash, trial_duration, 
                                                  bin_size)
    # make xarray
    session.trial_da = xr.DataArray(trial_tensor, dims=("unit_id", "time", "trials"), 
                               coords={
                                   "unit_id": session.good_units.index.values,
                                   "time": np.arange(0, trial_duration, bin_size)-time_before_flash,
                                   "trials": session.trials.index.values
                                   })
    
    #shorten the area names to better lump together units
    #get rig of layers and/or sub-areas with dashes
    area_short = []
    for area in session.good_units['area']:
        if area=='N/A':
            short='N/A'
        elif area[:2]=='CA':
            short=area
        else:
            dig_ind=re.search(r"\d", area)
            dash_ind=re.search(r"-", area)
            if dig_ind!=None:
                short=area[:dig_ind.start()]
            elif dash_ind!=None:
                short=area[:dash_ind.start()]
            else:
                short=area
        area_short.append(short)
    session.good_units['area_short']=area_short
    area_counts=session.good_units['area_short'].value_counts()
    
    #try more timepoints & split by areas
    svc_results={}
    svc_results_shuff={}

    # grab data: make these [trials,neurons] matrices based on different time bins:

#     hit_trials = session.trials.query(
#         '(trialStimID == "vis1" and \
#         trialstimRewarded == "vis1" and \
#         trial_response == True and \
#         vis_autoreward_trials == False) or \
#         (trialStimID == "sound1" and \
#         trialstimRewarded == "sound1" and \
#         trial_response == True and \
#         vis_autoreward_trials == False)').index

#     cr_trials = session.trials.query(
#         '(trialStimID == "vis1" and \
#         trialstimRewarded == "sound1" and \
#         trial_response == False and \
#         vis_autoreward_trials == False) or \
#         (trialStimID == "sound1" and \
#         trialstimRewarded == "vis1" and \
#         trial_response == False and \
#         vis_autoreward_trials == False)').index


#     trial_sel=np.sort(np.hstack([hit_trials,cr_trials]))
    
    if 'trialOptoVoltage' in session.trials.columns:
        trial_sel = session.trials.query('vis_autoreward_trials == False and \
                                          aud_autoreward_trials == False and \
                                          cross_modal_dprime >= 1.5 and \
                                          trialStimID != "catch" and \
                                          trialOptoVoltage.isnull()').index
    else:
        trial_sel = session.trials.query('vis_autoreward_trials == False and \
                                          aud_autoreward_trials == False and \
                                          cross_modal_dprime >= 1.5 and \
                                          trialStimID != "catch"').index
    

    # grab the stimulus ids
    stim_ids = session.trials['trialStimID'][trial_sel].values
    # or, use block IDs
    block_ids = session.trials['trialstimRewarded'][trial_sel].values
    #or, use whether mouse responded
    trial_response = session.trials['trial_response'][trial_sel].values

    #choose what variable to predict
    pred_var = block_ids

#     ### make sure equal # of trials per condition!
#     conds = np.unique(pred_var)
#     cond_count = np.zeros(len(conds))
#     for ic,cc in enumerate(conds):
#         cond_count[ic]=np.sum(pred_var==cc)
#     min_n_trials=np.min(cond_count).astype(int)
#     subset_ind=[]
#     for cc in conds:
#         cond_inds=np.where(pred_var==cc)[0]
#         subset_ind.append(np.random.choice(cond_inds,min_n_trials,replace=False))   
#     subset_ind=np.sort(np.hstack(subset_ind))
#     trial_sel=trial_sel[subset_ind]
#     pred_var=pred_var[subset_ind]


    area_sel = area_counts[area_counts>=u_num].index
    # area_sel = ['all']

    for aa in area_sel:
        if aa=='all':
            unit_sel = session.good_units.index.values
        else:
            unit_sel = session.good_units.query('area_short==@aa').index.values
        svc_results[aa]={}
        svc_results_shuff[aa]={}

        for tt,t_start in enumerate(time_bins[:-1]):
            svc_results[aa][tt]={}
            svc_results_shuff[aa][tt]={}

            for nn in range(0,n_repeats):
                unit_subset = np.random.choice(unit_sel,u_num,replace=True)

                ### make sure equal # of trials per condition!
                subset_ind=[]
                conds = np.unique(pred_var)
                cond_count=[]

                for cc in conds:
                    cond_inds=np.where(pred_var==cc)[0]
                    if len(cond_inds)<trnum:
                        trnum=len(cond_inds)
                    subset_ind.append(np.random.choice(cond_inds,trnum,replace=False))   
                subset_ind=np.sort(np.hstack(subset_ind))

                sel_data = session.trial_da.sel(time=slice(t_start,time_bins[tt+1]),
                                                trials=trial_sel[subset_ind],
                                                unit_id=unit_subset).mean(dim='time').values
                scaler = StandardScaler()
                scaler.fit(sel_data.T)
                X = scaler.transform(sel_data.T)
#                 X = sel_data.T
                y = pred_var[subset_ind].flatten()
                
                xtrain, xtest, ytrain, ytest = train_test_split(X, y, test_size=0.5, stratify=y)

                clf = svm.LinearSVC()
                clf.fit(xtrain, ytrain)

                ypred = clf.predict(xtest)

                cr_dict=classification_report(ytest, ypred, output_dict=True)
                cr_df=pd.DataFrame.from_dict(cr_dict)

                svc_results[aa][tt][nn]=cr_df
                
                #shuffle labels & train
                y_shuff = np.random.choice(y,len(y),replace=False)
                
                xtrain_shuff, xtest_shuff, ytrain_shuff, ytest_shuff = train_test_split(X, y_shuff, test_size=0.5, stratify=y)

                clf = svm.LinearSVC()
                clf.fit(xtrain_shuff, ytrain_shuff)

                ypred_shuff = clf.predict(xtest_shuff)

                cr_dict_shuff=classification_report(ytest_shuff, ypred_shuff, output_dict=True)
                cr_df_shuff=pd.DataFrame.from_dict(cr_dict_shuff)

                svc_results_shuff[aa][tt][nn]=cr_df_shuff
                
        print(aa+' done')
        
        
    # make into more plottable format
    plot_results={}
    timepoints=time_bins[1:]
    areas=area_sel

    for aa in areas:
        plot_results[aa]={}
        plot_results[aa]['vis1']=np.zeros((len(timepoints),n_repeats))
        plot_results[aa]['vis1'][:]=np.nan
        plot_results[aa]['aud1']=np.zeros((len(timepoints),n_repeats))
        plot_results[aa]['aud1'][:]=np.nan
        
        plot_results[aa]['True']=np.zeros((len(timepoints),n_repeats))
        plot_results[aa]['True'][:]=np.nan
        plot_results[aa]['False']=np.zeros((len(timepoints),n_repeats))
        plot_results[aa]['False'][:]=np.nan

    # diff table for each timepoint?
    # row = session
    # columns = ['modality','A_vis','A_aud','B_vis','B_aud','C_vis','C_aud','F_vis','F_aud',]

    for aa in areas: #svc_results[ss]['results'].keys():
        if aa in svc_results.keys():
            for tt,tp in enumerate(timepoints):

                for nn in range(0,n_repeats):
                    if 'vis1' in svc_results[aa][tt][nn].keys():
                        vis1_perf=np.mean(svc_results[aa][tt][nn]['vis1'][['precision','recall']].values)
                        plot_results[aa]['vis1'][tt,nn]=vis1_perf

                    if 'sound1' in svc_results[aa][tt][nn].keys():
                        aud1_perf=np.mean(svc_results[aa][tt][nn]['sound1'][['precision','recall']].values)
                        plot_results[aa]['aud1'][tt,nn]=aud1_perf
                        
                    if 'True' in svc_results[aa][tt][nn].keys():
                        vis1_perf=np.mean(svc_results[aa][tt][nn]['True'][['precision','recall']].values)
                        plot_results[aa]['True'][tt,nn]=vis1_perf

                    if 'False' in svc_results[aa][tt][nn].keys():
                        aud1_perf=np.mean(svc_results[aa][tt][nn]['False'][['precision','recall']].values)
                        plot_results[aa]['False'][tt,nn]=aud1_perf
    
    
    
    fig,ax=plt.subplots(1,1,figsize=(6,4))

    for aa,area in enumerate(plot_results.keys()):
        y=np.nanmean([np.mean(plot_results[area]['vis1'],1),np.mean(plot_results[area]['aud1'],1)],0)
        yerr=np.std(np.hstack([plot_results[area]['vis1'],plot_results[area]['aud1']]),1)/np.sqrt(n_repeats)
        
#         y=np.nanmean([np.mean(plot_results[area]['True'],1),np.mean(plot_results[area]['False'],1)],0)
#         yerr=np.std(np.hstack([plot_results[area]['True'],plot_results[area]['False']]),1)/np.sqrt(n_repeats)
        
        if aa>19:
            linex=ax.plot(timepoints,y,linestyle='-.')
        elif aa>9:
            linex=ax.plot(timepoints,y,linestyle='--')
        else:
            linex=ax.plot(timepoints,y)
        plt.fill_between(timepoints, y-yerr, y+yerr,
                        alpha=0.2, edgecolor=None, facecolor=linex[0].get_color())

    ax.set_ylim([0.4,1.0])
    ax.axvline(0,color='k')
    ax.axhline(0.5,color='k',linewidth=0.5,linestyle='--')
    ax.set_title(session.metadata['mouseID']+' session '+str(session.metadata['ephys_session_num'])+
                '; n_units='+str(u_num)+'; n_trials='+str(trnum)+'; n_repeats='+str(n_repeats))
    ax.legend(plot_results.keys(),ncol=3)
    ax.set_ylabel('decodability of context')
    ax.set_xlabel('time from stimulus onset (s)')
    fig.tight_layout()
    
    figpath=r"C:\Users\ethan.mcbride\OneDrive - Allen Institute\quick figures\2023-04-04-DR_SAC_decoding_context"

    figname=(session.metadata['mouseID']+'_session_'+str(session.metadata['ephys_session_num'])+
                '_n_units_'+str(u_num)+'_n_trials_'+str(trnum)+'_n_repeats_'+str(n_repeats)+'.png')
    # figname='control_heatmap_500ms_all.png'
    plt.savefig(os.path.join(figpath,figname), dpi=600, facecolor='w', edgecolor='w',
                orientation='portrait', format='png',
                transparent=True, bbox_inches='tight', pad_inches=0.1,
                metadata=None)
    
    ### shuffled labels plot
    # make into more plottable format
    plot_results_shuff={}
    timepoints=time_bins[1:]
    areas=area_sel

    for aa in areas:
        plot_results_shuff[aa]={}
        plot_results_shuff[aa]['vis1']=np.zeros((len(timepoints),n_repeats))
        plot_results_shuff[aa]['vis1'][:]=np.nan
        plot_results_shuff[aa]['aud1']=np.zeros((len(timepoints),n_repeats))
        plot_results_shuff[aa]['aud1'][:]=np.nan
        
        plot_results_shuff[aa]['True']=np.zeros((len(timepoints),n_repeats))
        plot_results_shuff[aa]['True'][:]=np.nan
        plot_results_shuff[aa]['False']=np.zeros((len(timepoints),n_repeats))
        plot_results_shuff[aa]['False'][:]=np.nan

    # diff table for each timepoint?
    # row = session
    # columns = ['modality','A_vis','A_aud','B_vis','B_aud','C_vis','C_aud','F_vis','F_aud',]

    for aa in areas: #svc_results[ss]['results'].keys():
        if aa in svc_results_shuff.keys():
            for tt,tp in enumerate(timepoints):

                for nn in range(0,n_repeats):
                    if 'vis1' in svc_results_shuff[aa][tt][nn].keys():
                        vis1_perf=np.mean(svc_results_shuff[aa][tt][nn]['vis1'][['precision','recall']].values)
                        plot_results_shuff[aa]['vis1'][tt,nn]=vis1_perf

                    if 'sound1' in svc_results_shuff[aa][tt][nn].keys():
                        aud1_perf=np.mean(svc_results_shuff[aa][tt][nn]['sound1'][['precision','recall']].values)
                        plot_results_shuff[aa]['aud1'][tt,nn]=aud1_perf
                        
                    if 'True' in svc_results_shuff[aa][tt][nn].keys():
                        vis1_perf=np.mean(svc_results_shuff[aa][tt][nn]['True'][['precision','recall']].values)
                        plot_results_shuff[aa]['True'][tt,nn]=vis1_perf

                    if 'False' in svc_results_shuff[aa][tt][nn].keys():
                        aud1_perf=np.mean(svc_results_shuff[aa][tt][nn]['False'][['precision','recall']].values)
                        plot_results_shuff[aa]['False'][tt,nn]=aud1_perf
    
    
    
    fig,ax=plt.subplots(1,1,figsize=(6,4))

    for aa,area in enumerate(plot_results.keys()):
        y=np.nanmean([np.mean(plot_results_shuff[area]['vis1'],1),np.mean(plot_results_shuff[area]['aud1'],1)],0)
        yerr=np.std(np.hstack([plot_results_shuff[area]['vis1'],plot_results_shuff[area]['aud1']]),1)/np.sqrt(n_repeats)
        
#         y=np.nanmean([np.mean(plot_results[area]['True'],1),np.mean(plot_results[area]['False'],1)],0)
#         yerr=np.std(np.hstack([plot_results[area]['True'],plot_results[area]['False']]),1)/np.sqrt(n_repeats)
        
        if aa>19:
            linex=ax.plot(timepoints,y,linestyle='-.')
        elif aa>9:
            linex=ax.plot(timepoints,y,linestyle='--')
        else:
            linex=ax.plot(timepoints,y)
        plt.fill_between(timepoints, y-yerr, y+yerr,
                        alpha=0.2, edgecolor=None, facecolor=linex[0].get_color())

    ax.set_ylim([0.4,1.0])
    ax.axvline(0,color='k')
    ax.axhline(0.5,color='k',linewidth=0.5,linestyle='--')
    ax.set_title(session.metadata['mouseID']+' session '+str(session.metadata['ephys_session_num'])+
                '; n_units='+str(u_num)+'; n_trials='+str(trnum)+'; n_repeats='+str(n_repeats)+' shuffled')
    ax.legend(plot_results.keys(),ncol=3)
    ax.set_ylabel('decodability of context')
    ax.set_xlabel('time from stimulus onset (s)')
    fig.tight_layout()
    
    figpath=r"C:\Users\ethan.mcbride\OneDrive - Allen Institute\quick figures\2023-04-04-DR_SAC_decoding_context"

    figname=(session.metadata['mouseID']+'_session_'+str(session.metadata['ephys_session_num'])+
                '_n_units_'+str(u_num)+'_n_trials_'+str(trnum)+'_n_repeats_'+str(n_repeats)+'_shuffled.png')
    # figname='control_heatmap_500ms_all.png'
    plt.savefig(os.path.join(figpath,figname), dpi=600, facecolor='w', edgecolor='w',
                orientation='portrait', format='png',
                transparent=True, bbox_inches='tight', pad_inches=0.1,
                metadata=None)
    
    print(path+' done!')


In [ ]:
cond_inds.shape

In [ ]:
unit_sel

In [ ]:
session.good_units.query('area_short==@aa').index.values

In [ ]:
session.good_units['area_short'].unique()

## loop through experiments & save

In [ ]:
trnum='all'
u_num='all'
u_min=20
n_repeats=100
binsize=0.1
time_bins=np.arange(-0.2,0.6,binsize)
        
svc_results={}

for sel_session, path in enumerate(main_path):
    
    svc_results[sel_session]={}
    
    session=[]
    session=Session(path=path) 
    session.assign_unit_areas()
    session=compute_smoothed_response_rate(session)
    
    # loop through sessions and make unit xarrays
    time_before_flash = 0.25
    trial_duration = 1.0
    bin_size = 0.001

    # Make tensor (3-D matrix [units,time,trials])
    trial_tensor = make_neuron_time_trials_tensor(session.good_units, session.spike_times, 
                                                  session.trials,time_before_flash, trial_duration, 
                                                  bin_size)
    # make xarray
    session.trial_da = xr.DataArray(trial_tensor, dims=("unit_id", "time", "trials"), 
                               coords={
                                   "unit_id": session.good_units.index.values,
                                   "time": np.arange(0, trial_duration, bin_size)-time_before_flash,
                                   "trials": session.trials.index.values
                                   })
    
    #shorten the area names to better lump together units
    #get rig of layers and/or sub-areas with dashes
    area_short = []
    for area in session.good_units['area']:
        if area=='N/A':
            short='N/A'
        elif area[:2]=='CA':
            short=area
        else:
            dig_ind=re.search(r"\d", area)
            dash_ind=re.search(r"-", area)
            if dig_ind!=None:
                short=area[:dig_ind.start()]
            elif dash_ind!=None:
                short=area[:dash_ind.start()]
            else:
                short=area
        area_short.append(short)
    session.good_units['area_short']=area_short
    area_counts=session.good_units['area_short'].value_counts()

    # grab data: make these [trials,neurons] matrices based on different time bins:

#     hit_trials = session.trials.query(
#         '(trialStimID == "vis1" and \
#         trialstimRewarded == "vis1" and \
#         trial_response == True and \
#         vis_autoreward_trials == False and \ 
#         aud_autoreward_trials == False and \
#         cross_modal_dprime >= 1.5 or \
#         (trialStimID == "sound1" and \
#         trialstimRewarded == "sound1" and \
#         trial_response == True and \
#         vis_autoreward_trials == False and \ 
#         aud_autoreward_trials == False and \
#         cross_modal_dprime >= 1.5').index

#     cr_trials = session.trials.query(
#         '(trialStimID == "vis1" and \
#         trialstimRewarded == "sound1" and \
#         trial_response == False and \
#         vis_autoreward_trials == False and \ 
#         aud_autoreward_trials == False and \
#         cross_modal_dprime >= 1.5 or \
#         (trialStimID == "sound1" and \
#         trialstimRewarded == "vis1" and \
#         trial_response == False and \
#         vis_autoreward_trials == False and \ 
#         aud_autoreward_trials == False and \
#         cross_modal_dprime >= 1.5').index

    
#     trial_sel=np.sort(np.hstack([hit_trials,cr_trials]))
    
    if 'trialOptoVoltage' in session.trials.columns:
        trial_sel = session.trials.query('vis_autoreward_trials == False and \
                                          aud_autoreward_trials == False and \
                                          cross_modal_dprime >= 1.5 and \
                                          trialStimID != "catch" and \
                                          trialOptoVoltage.isnull()').index
    else:
        trial_sel = session.trials.query('vis_autoreward_trials == False and \
                                          aud_autoreward_trials == False and \
                                          cross_modal_dprime >= 1.5 and \
                                          trialStimID != "catch"').index
        
    predict=['stim_ids','block_ids','trial_response']

    # grab the stimulus ids
    stim_ids = session.trials['trialStimID'][trial_sel].values
    # or, use block IDs
    block_ids = session.trials['trialstimRewarded'][trial_sel].values
    #or, use whether mouse responded
    trial_response = session.trials['trial_response'][trial_sel].values
      
    for p in predict:
        svc_results[sel_session][p]={}
    
        #choose what variable to predict
        if p=='stim_ids':
            pred_var = stim_ids
        elif p=='block_ids':
            pred_var = block_ids
        elif p=='trial_response':
            pred_var = trial_response

        area_sel = area_counts[area_counts>=u_min].index
        # area_sel = ['all']

        for aa in area_sel:
            if aa=='all':
                unit_sel = session.good_units.index.values
            else:
                unit_sel = session.good_units.query('area_short==@aa').index.values
            svc_results[sel_session][p][aa]={}

            for tt,t_start in enumerate(time_bins[:-1]):
                svc_results[sel_session][p][aa][tt]={}

                for nn in range(0,n_repeats):
                    if u_num=='all':
                        unit_subset = np.random.choice(unit_sel,len(unit_sel),replace=True)
                    else:
                        unit_subset = np.random.choice(unit_sel,u_num,replace=True)
                        
                    ### make sure equal # of trials per condition!
                    subset_ind=[]
                    conds = np.unique(pred_var)
                    cond_count=[]
                    
                    if trnum=='all':
                        for cc in conds:
                            cond_count.append(np.sum(pred_var==cc))
                        use_trnum=np.min(cond_count)
                    else:
                        use_trnum = trnum
                    
                    for cc in conds:
                        cond_inds=np.where(pred_var==cc)[0]
                        if len(cond_inds)<use_trnum:
                            use_trnum=len(cond_inds)
                        subset_ind.append(np.random.choice(cond_inds,use_trnum,replace=False))   
                    subset_ind=np.sort(np.hstack(subset_ind))

                    sel_data = session.trial_da.sel(time=slice(t_start,time_bins[tt+1]),
                                                    trials=trial_sel[subset_ind],
                                                    unit_id=unit_subset).mean(dim='time').values

#                     X = sel_data.T
                    scaler = StandardScaler()
                    scaler.fit(sel_data.T)
                    X = scaler.transform(sel_data.T)
                    y = pred_var[subset_ind].flatten()

                    xtrain, xtest, ytrain, ytest = train_test_split(X, y, test_size=0.2, stratify=y)

                    clf = svm.LinearSVC()
                    clf.fit(xtrain, ytrain)

                    ypred = clf.predict(xtest)

                    cr_dict=classification_report(ytest, ypred, output_dict=True)
                    cr_df=pd.DataFrame.from_dict(cr_dict)

                    svc_results[sel_session][p][aa][tt][nn]=cr_df

            print(aa+' done')

            
savepath=r'C:\Users\ethan.mcbride\OneDrive - Allen Institute\DR decoding results'

with open(os.path.join(savepath,'decoder_results_100ms_incl_all_trials.pkl'), 'wb') as handle:
    pickle.dump(svc_results, handle, protocol=pickle.HIGHEST_PROTOCOL)        

In [ ]:
# svc_results[0]['block_ids'].keys()

In [ ]:
# svc_results[5]['block_ids']['MOs'][0][0]['vis1']

In [ ]:
# u_num

In [ ]:
# make into more plottable format

####change this to deal with multiple #s of trials and average over re-runs of the SVC

plot_results={}

timepoints=time_bins[1:]

label_list=['vis1','vis2','sound1','sound2','True','False']
# label_list=['vis1','sound1','True','False']

predict=['stim_ids','block_ids','trial_response']

areas=['VISp','AUD','MOs','MRN','CP']

for sel_session, path in enumerate(main_path):
    plot_results[sel_session]={}
    for aa in areas:
        plot_results[sel_session][aa]={}
        for p in predict:
            plot_results[sel_session][aa][p]={}
            for ll in label_list:
                plot_results[sel_session][aa][p][ll]=np.zeros((len(timepoints),n_repeats))
                plot_results[sel_session][aa][p][ll][:]=np.nan

    
# diff table for each timepoint?
# row = session
# columns = ['modality','A_vis','A_aud','B_vis','B_aud','C_vis','C_aud','F_vis','F_aud',]

decoder_acc_session_mean = {}
for aa in areas:
    decoder_acc_session_mean[aa]={}
    for p in predict:
        decoder_acc_session_mean[aa][p]={}
        for ll in label_list:
            
            decoder_acc_session_mean[aa][p][ll]=np.zeros((len(timepoints),
                                                           len(main_path)))
            decoder_acc_session_mean[aa][p][ll][:]=np.nan
        

for sel_session, path in enumerate(main_path):
    for p in predict:
        for aa in areas: 
            if aa in svc_results[sel_session][p].keys():
                for tt,tp in enumerate(timepoints):
                    for nn in range(0,n_repeats):
                        for ll in label_list:
                            if len(svc_results[sel_session][p][aa])>0:
                                if len(svc_results[sel_session][p][aa][tt])>0:
                                    if ll in svc_results[sel_session][p][aa][tt][nn].keys():
                                        temp_perf=np.nanmean(svc_results[sel_session][p]
                                                          [aa][tt][nn][ll]
                                                          [['precision','recall']].values)
                                        plot_results[sel_session][aa][p][ll][tt,nn]=temp_perf

                    for ll in label_list:
                        if len(svc_results[sel_session][p][aa])>0:                    
#                             concat_results = np.concatenate((plot_results[sel_session][aa][p][ll][tt],
#                                                              plot_results[sel_session][aa][p][ll][tt]),
#                                                             axis=2)
#                             decoder_acc_mean = np.nanmean(concat_results,2)
                            decoder_acc_mean = np.nanmean(plot_results[sel_session][aa][p][ll][tt])
                            decoder_acc_session_mean[aa][p][ll][tt,sel_session] = decoder_acc_mean
    
    
# plot_results[area][label/stimulus][timepoint_idx][ntrials_idx,nunits_idx,n_repeats]

In [ ]:
# plot_results[sel_session][aa][p][ll][tt]
plot_results[sel_session].keys()

In [ ]:
np.nanmean(decoder_acc_session_mean['VISp']['block_ids']['vis1'],axis=1)

In [ ]:
predict_var='block_ids'
sel_stim='sound1'

fig,ax=plt.subplots(1,1)

ax.axvline(0,color='k',linestyle='--',linewidth=1)
ax.axhline(0.5,color='k',linestyle='--',linewidth=1)

ax.plot(time_bins[1:],decoder_acc_session_mean['VISp'][predict_var][sel_stim],'k',linewidth=0.75,alpha=0.5)
# ax.plot(time_bins[1:],decoder_acc_session_mean['AUDp'][predict_var][sel_stim],'m',linewidth=0.75,alpha=0.5)
ax.plot(time_bins[1:],decoder_acc_session_mean['MOs'][predict_var][sel_stim],'b',linewidth=0.75,alpha=0.5)
# ax.plot(time_bins[1:],decoder_acc_session_mean['MRN'][predict_var][sel_stim],'r',linewidth=0.75,alpha=0.5)
# ax.plot(time_bins[1:],decoder_acc_session_mean['CP'][predict_var][sel_stim],'g',linewidth=0.75,alpha=0.5)


visp_line=ax.plot(time_bins[1:],np.nanmean(decoder_acc_session_mean['VISp'][predict_var][sel_stim],axis=1)
        ,'k.-',linewidth=2)
# ax.plot(time_bins[1:],np.nanmean(decoder_acc_session_mean['AUDp'][predict_var][sel_stim],axis=1)
#         ,'m.-',linewidth=2)
mos_line=ax.plot(time_bins[1:],np.nanmean(decoder_acc_session_mean['MOs'][predict_var][sel_stim],axis=1)
        ,'b.-',linewidth=2)
# ax.plot(time_bins[1:],np.nanmean(decoder_acc_session_mean['MRN'][predict_var][sel_stim],axis=1)
#         ,'r.-',linewidth=2)
# ax.plot(time_bins[1:],np.nanmean(decoder_acc_session_mean['CP'][predict_var][sel_stim],axis=1)
#         ,'g.-',linewidth=2)


ax.set_ylim([0.4,1.0])
ax.set_xlabel('time relative to stim onset (s)')
ax.set_ylabel('block decoding accuracy')
ax.set_title('decode: '+sel_stim)
ax.legend([visp_line[0],mos_line[0]],['VISp','MOs'])

In [ ]:
decoder_acc_session_mean['AUD']

In [ ]:
predict_var='block_ids'
sel_stim='vis1'
st.ranksums(decoder_acc_session_mean['VISp'][predict_var][sel_stim][0,:],
            decoder_acc_session_mean['MOs'][predict_var][sel_stim][0,:],
            nan_policy='omit')

In [ ]:
np.sum(~np.isnan(decoder_acc_session_mean['MOs'][predict_var][sel_stim][0,:]))

## loop through experiments, save all relevant info (plot later)

In [ ]:
def custom_decoder(input_data,labels):
    
    output={}
    
    scaler = StandardScaler()
    skf = StratifiedKFold(n_splits=5,shuffle=True)
    
    scaler.fit(sel_data.T)
    X = scaler.transform(input_data)
    y = labels
 
    if type(y[0])==bool:
        ypred=np.full(len(y), fill_value=False)
    elif type(y[0])==str:
        ypred=np.full(len(y), fill_value='       ')
    else:
        ypred=np.full(len(y), fill_value=np.nan)

    tidx_used=[]

    for train,test in skf.split(X, y):
        clf=svm.LinearSVC()
        clf.fit(X[train],y[train])
        ypred[test] = clf.predict(X[test])
        tidx_used.append([test])

    cr_dict=classification_report(y, ypred, output_dict=True)
    cr_df=pd.DataFrame.from_dict(cr_dict)

    output['cr']=cr_df
    output['pred_label']=ypred
    output['true_label']=y
    output['trials_used']=tidx_used
    
    return output


In [ ]:
trnum='all'
u_num='all'
u_min=20
n_repeats=100
binsize=0.1
time_bins=np.arange(-0.5,1.0,binsize)
balance_labels=0

svc_results={}

for sel_session, path in enumerate(main_path):
    
    svc_results[sel_session]={}
    
    session=[]
    session=Session(path=path) 
    session.assign_unit_areas()
    session=compute_smoothed_response_rate(session)
    
    # loop through sessions and make unit xarrays
    time_before_flash = 0.5
    trial_duration = 1.5
    bin_size = 0.001

    # Make tensor (3-D matrix [units,time,trials])
    trial_tensor = make_neuron_time_trials_tensor(session.good_units, session.spike_times, 
                                                  session.trials,time_before_flash, trial_duration, 
                                                  bin_size)
    # make xarray
    session.trial_da = xr.DataArray(trial_tensor, dims=("unit_id", "time", "trials"), 
                               coords={
                                   "unit_id": session.good_units.index.values,
                                   "time": np.arange(0, trial_duration, bin_size)-time_before_flash,
                                   "trials": session.trials.index.values
                                   })
    
    #shorten the area names to better lump together units
    #get rid of layers and/or sub-areas with dashes
    area_short = []
    for area in session.good_units['area']:
        if area=='N/A':
            short='N/A'
        elif area[:2]=='CA':
            short=area
        else:
            dig_ind=re.search(r"\d", area)
            dash_ind=re.search(r"-", area)
            if dig_ind!=None:
                short=area[:dig_ind.start()]
            elif dash_ind!=None:
                short=area[:dash_ind.start()]
            else:
                short=area
        area_short.append(short)
    session.good_units['area_short']=area_short
    area_counts=session.good_units['area_short'].value_counts()

    #exclude any trials that had opto stimulation
    if 'trialOptoVoltage' in session.trials.columns:
        trial_sel = session.trials.query('trialOptoVoltage.isnull()').index
    else:
        trial_sel = session.trials.index

    predict=['stim_ids','block_ids','trial_response']

    # grab the stimulus ids
    stim_ids = session.trials['trialStimID'][trial_sel].values
    # or, use block IDs
    block_ids = session.trials['trialstimRewarded'][trial_sel].values
    #or, use whether mouse responded
    trial_response = session.trials['trial_response'][trial_sel].values
    
    #save metadata about this session & decoder params
    svc_results[sel_session]['metadata']=session.metadata
    svc_results[sel_session]['trial_numbers']=trnum
    svc_results[sel_session]['unit_numbers']=u_num
    svc_results[sel_session]['min_n_units']=u_min
    svc_results[sel_session]['n_repeats']=n_repeats
    svc_results[sel_session]['time_bins']=time_bins
    svc_results[sel_session]['balance_labels']=balance_labels
    
    #loop through different labels to predict
    for p in predict:
        svc_results[sel_session][p]={}
    
        #choose what variable to predict
        if p=='stim_ids':
            pred_var = stim_ids
        elif p=='block_ids':
            pred_var = block_ids
        elif p=='trial_response':
            pred_var = trial_response

        area_sel = area_counts[area_counts>=u_min].index
        # area_sel = ['all']
        
        #loop through areas
        for aa in area_sel:
            if aa=='all':
                unit_sel = session.good_units.index.values
            else:
                unit_sel = session.good_units.query('area_short==@aa').index.values
            svc_results[sel_session][p][aa]={}
            svc_results[sel_session][p][aa]['n_units']=len(unit_sel)
            
            #loop through time bins
            for tt,t_start in enumerate(time_bins[:-1]):
                svc_results[sel_session][p][aa][tt]={}
                
                #loop through repeats
                for nn in range(0,n_repeats):
                    
                    if u_num=='all':
                        unit_subset = np.random.choice(unit_sel,len(unit_sel),replace=False)
                    else:
                        unit_subset = np.random.choice(unit_sel,u_num,replace=False)
                    
                    #option to balance number of labels for training
                    if balance_labels:
                        subset_ind=[]
                        conds = np.unique(pred_var)
                        cond_count=[]

                        if trnum=='all':
                            for cc in conds:
                                cond_count.append(np.sum(pred_var==cc))
                            use_trnum=np.min(cond_count)
                        else:
                            use_trnum = trnum

                        for cc in conds:
                            cond_inds=np.where(pred_var==cc)[0]
                            if len(cond_inds)<use_trnum:
                                use_trnum=len(cond_inds)
                            subset_ind.append(np.random.choice(cond_inds,use_trnum,replace=False))   
                        subset_ind=np.sort(np.hstack(subset_ind))

                        trial_sel=trial_sel[subset_ind]
                        pred_var=pred_var[subset_ind]


                    sel_data = session.trial_da.sel(time=slice(t_start,time_bins[tt+1]),
                                                    trials=trial_sel,
                                                    unit_id=unit_subset).mean(dim='time').values

                    svc_results[sel_session][p][aa][tt][nn]=custom_decoder(
                        input_data=sel_data.T,
                        labels=pred_var.flatten())

                    svc_results[sel_session][p][aa][tt][nn]['shuffle']=custom_decoder(
                        input_data=sel_data.T,
                        labels=np.random.choice(pred_var,len(pred_var),replace=False).flatten())

                    svc_results[sel_session][p][aa][tt][nn]['trial_sel_idx']=trial_sel
                    

            print(aa+' done')
            
    print(path+' done')
            
savepath=r'C:\Users\ethan.mcbride\OneDrive - Allen Institute\DR decoding results'

with open(os.path.join(savepath,'decoder_results_100ms_incl_all_units_trials_metadata.pkl'), 'wb') as handle:
    pickle.dump(svc_results, handle, protocol=pickle.HIGHEST_PROTOCOL)        